In [19]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import deque
import random
import os

In [20]:
# Hyperparameters
GAMMA = 0.99
TAU = 0.005
LR_ACTOR = 0.0001
LR_CRITIC = 0.001
BUFFER_SIZE = 1000000
BATCH_SIZE = 64
NOISE_DECAY = 0.99
MAX_EPISODES = 300
MAX_TIMESTEPS = 2000
# SAVE_PATH = "/home/janak/Documents/Pytorch_CPU/DDPG/DDPG/bipedal/Downloads/Hardcore/ddpg_bipedalwalker"
SAVE_PATH = "/home/janak/Documents/Pytorch_CPU/DDPG/DDPG/bipedal/Downloads/GPU/ddpg_bipedalwalker"



In [21]:
# Set the random seed for reproducibility
SEED = 32
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
env = gym.make('BipedalWalker-v3')
env = gym.make('BipedalWalker-v3', hardcore=True)
env.seed(SEED)

[32]

In [22]:
# Actor Network
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, action_dim)
        self.max_action = max_action

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x * self.max_action

# Critic Network
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, 1)

    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Replay Buffer
class ReplayBuffer:
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

    def size(self):
        return len(self.buffer)

# DDPG Agent
class DDPGAgent:
    def __init__(self, state_dim, action_dim, max_action):
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=LR_ACTOR)

        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=LR_CRITIC)

        self.replay_buffer = ReplayBuffer(BUFFER_SIZE)
        self.max_action = max_action
        self.noise = 0.1

    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()

    def train(self):
        if self.replay_buffer.size() < BATCH_SIZE:
            return

        states, actions, rewards, next_states, dones = self.replay_buffer.sample(BATCH_SIZE)

        states = torch.FloatTensor(states).to(device)
        actions = torch.FloatTensor(actions).to(device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
        next_states = torch.FloatTensor(next_states).to(device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(device)

        # Critic loss
        target_actions = self.actor_target(next_states)
        target_q = self.critic_target(next_states, target_actions)
        target_q = rewards + ((1 - dones) * GAMMA * target_q).detach()

        current_q = self.critic(states, actions)
        critic_loss = nn.MSELoss()(current_q, target_q)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Actor loss
        actor_loss = -self.critic(states, self.actor(states)).mean()

        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update target networks
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
            target_param.data.copy_(TAU * param.data + (1 - TAU) * target_param.data)

        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(TAU * param.data + (1 - TAU) * target_param.data)

    def add_to_replay_buffer(self, state, action, reward, next_state, done):
        self.replay_buffer.add(state, action, reward, next_state, done)

    def save(self, filename):
        torch.save(self.actor.state_dict(), filename + "_actor.pth")
        torch.save(self.critic.state_dict(), filename + "_critic.pth")

    def load(self, filename):
        self.actor.load_state_dict(torch.load(filename + "_actor.pth",map_location=device))
        self.critic.load_state_dict(torch.load(filename + "_critic.pth",map_location=device))



In [23]:
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
agent = DDPGAgent(state_dim, action_dim, max_action)

In [25]:
# Loading the model
agent.load(SAVE_PATH)

FileNotFoundError: [Errno 2] No such file or directory: '/home/janak/Documents/Pytorch_CPU/DDPG/DDPG/bipedal/Downloads/GPU/ddpg_bipedalwalker/Hardcore_actor.pth'

In [ ]:
import time

In [ ]:
# To test the loaded model:
for episode in range(10):
    state = env.reset()
    episode_reward = 0
    for t in range(MAX_TIMESTEPS):
        env.render()
        action = agent.select_action(state)
        # print(action)
        next_state, reward, done, _ = env.step(action)
        state = next_state
        episode_reward += reward
        if done:
            break
    print(f"Test Episode: {episode}, Reward: {episode_reward}")
    time.sleep(2)
env.close()

Test Episode: 0, Reward: -96.81204132838174
Test Episode: 1, Reward: -90.48696249828612
Test Episode: 2, Reward: -90.73712797848633
Test Episode: 3, Reward: -28.631914930904657
Test Episode: 4, Reward: -63.38290924780196
Test Episode: 5, Reward: -88.23407320822962
Test Episode: 6, Reward: -36.189504999298364
Test Episode: 7, Reward: -52.080410386965916
Test Episode: 8, Reward: -86.88270176064674
Test Episode: 9, Reward: -56.90193344512305
